In [1]:
%pip install llama-index-embeddings-openai
%pip install llama-index-llms-llama-cpp



     ---------------------------------------- 0.0/49.4 MB ? eta -:--:--
     ---------------------------------------- 0.0/49.4 MB ? eta -:--:--
     --------------------------------------- 0.0/49.4 MB 222.6 kB/s eta 0:03:42
     --------------------------------------- 0.0/49.4 MB 281.8 kB/s eta 0:02:55
     --------------------------------------- 0.1/49.4 MB 590.8 kB/s eta 0:01:24
     ---------------------------------------- 0.4/49.4 MB 1.8 MB/s eta 0:00:28
      --------------------------------------- 1.1/49.4 MB 4.4 MB/s eta 0:00:11
     -- ------------------------------------- 2.6/49.4 MB 8.9 MB/s eta 0:00:06
     --- ------------------------------------ 4.4/49.4 MB 13.4 MB/s eta 0:00:04
     ----- ---------------------------------- 6.3/49.4 MB 16.2 MB/s eta 0:00:03
     ------ --------------------------------- 8.1/49.4 MB 18.4 MB/s eta 0:00:03
     ------- -------------------------------- 9.5/49.4 MB 19.5 MB/s eta 0:00:03
     -------- ------------------------------ 11.3/49.4 MB 

In [2]:
!pip -q install llama-index python-dotenv pinecone-client


### Accessing vector database

If you havent uploaded data to a pinecone database, look at the ingesting_data notebook

In [15]:
import os 
from dotenv import load_dotenv
from pinecone import Pinecone
from llama_index.vector_stores.pinecone import PineconeVectorStore


load_dotenv()
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("harrison-medical-data")
vector_store = PineconeVectorStore(pinecone_index=index)


### Defining embedding

In [3]:
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model = OpenAIEmbedding()

In [8]:
query_str = "In 2 sentences tell me what is the power house of the cell"

In [10]:
embedded_query = embed_model.get_query_embedding(query_str)

In [16]:
from llama_index.core.vector_stores import VectorStoreQuery

query_mode = "default"
# query_mode = "sparse"
# query_mode = "hybrid"

vector_store_query = VectorStoreQuery(
    query_embedding=embedded_query, similarity_top_k=5, mode=query_mode
)

In [21]:
query_result = vector_store.query(vector_store_query)
print(query_result)

VectorStoreQueryResult(nodes=[TextNode(id_='6b4ce73c-ddd9-4707-a88a-59e7ba2b916a', embedding=[-0.0106595131, -0.00529331947, 0.00202722871, -0.037126109, -0.0096988976, 0.023452254, 0.00540594291, 0.00846666098, -0.0437510386, -0.00885090698, 0.0229090098, 0.0114876293, 0.00519725773, 0.0124614937, 0.00875153299, 0.000981317833, 0.0573983975, -0.0116135031, -0.0335751474, -0.0150518417, -0.0190135501, 0.0168935731, 0.00210507168, -0.00596574973, -0.0146013461, 0.0369671099, 0.0208420306, -0.0488919839, -0.00789691694, -0.00451157754, -0.00791016687, 0.00900990516, -0.0174500663, -0.0151578402, -0.0233595036, -0.00134320464, 0.0250422359, -0.0262214746, 0.00680380361, -0.00932127703, -0.013846104, 0.00493226061, -0.00355427503, 0.00353440037, -0.0116598774, -0.00129020517, 0.0149458423, -0.0208685305, -0.0144158481, 0.00953989942, 0.0164298266, 0.00995064527, -0.0149458423, 0.00714167487, -0.00947365072, -0.0245784912, 0.00239159982, 0.035589125, 0.00670774188, -0.00194441702, -0.010050

### Display results

In [37]:

from llama_index.core.schema import NodeWithScore
from typing import Optional

nodes_with_scores = []
for index, node in enumerate(query_result.nodes):
    score: Optional[float] = None
    if query_result.similarities is not None:
        score = query_result.similarities[index]
    nodes_with_scores.append(NodeWithScore(node=node, score=score))

In [39]:

# from llama_index.core.response.notebook_utils import display_source_node

# for node in nodes_with_scores:
#     display_source_node(node, source_length=1000)

### Putting everything together

In [46]:
from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from typing import Any, List


class PineconeRetriever(BaseRetriever):
    """Retriever over a pinecone vector store."""

    def __init__(
        self,
        vector_store: PineconeVectorStore,
        embed_model: Any,
        query_mode: str = "default",
        similarity_top_k: int = 2,
    ) -> None:
        """Init params."""
        self._vector_store = vector_store
        self._embed_model = embed_model
        self._query_mode = query_mode
        self._similarity_top_k = similarity_top_k
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        if query_bundle.embedding is None:
            query_embedding = self._embed_model.get_query_embedding(
                query_bundle.query_str
            )
        else:
            query_embedding = query_bundle.embedding

        vector_store_query = VectorStoreQuery(
            query_embedding=query_embedding,
            similarity_top_k=self._similarity_top_k,
            mode=self._query_mode,
        )
        query_result = self._vector_store.query(vector_store_query)

        nodes_with_scores = []
        for index, node in enumerate(query_result.nodes):
            score: Optional[float] = None
            if query_result.similarities is not None:
                score = query_result.similarities[index]
            nodes_with_scores.append(NodeWithScore(node=node, score=score))

        return nodes_with_scores

In [47]:

retriever = PineconeRetriever(
    vector_store, embed_model, query_mode="default", similarity_top_k=2
)